In [ ]:
from funman.funman import Funman
from funman.scenario import ParameterSynthesisScenario
from funman.model import Parameter
from funman.scenario import ParameterSynthesisScenarioResult
from funman.parameter_space import ParameterSpace
from funman.search_utils import SearchConfig

# %load_ext autoreload
# %autoreload 2

## Todo

- Jack: Caching of parameter space and reload
- Jack: Disable or fix realtime plotting in Jupyter
- Drisana: Merge notebook operations into classes
- Drisana: Test Generate a 4x1

In [ ]:
# The Funman module synthesizes the Parameter Space for a model, which encodes
# feasible parameter settings (denoted "true"), and infeasible parameter
# settings (denoted "false").  The code in this cell generates the Parameter
# Space and plots it.
#
# Funman synthesizes parameters with a box search algorithm that encodes a
# series of related Satisfiability Modulo Theories (SMT) encodings.  The box
# search involves sub-dividing the space of possible parameter-value
# combinations to identify boxes that either contain only feasible values or
# infeasible values for the parameters.  If a box contains a feasible point, the
# solver attempts to prove the absence of infeasible points.  If no infeasible
# points are found, then the box is labeled feasible.  Similarly, if a box
# contains an infeasible point and no feasible points, it is labeled infeasible.
# If a box contains both a feasible and infeasible point, then the solver
# subdivides the box.  The labeling and subdividing process continues until no
# boxes above a given threshold size remain.
#
# Evaluate the extracted model to identify values for beta that:
#  - Represent the intervention efficacy over two successive interventions
#  - Occur over two time periods 0-20 days (beta_1), and 20-40 days (beta_2)
#  - Prevent the number of infected I from exceeding 10% of the population
#  - Use a linearized definition for the S and R state update equations
#  - Fall within a specified range [0.0, 0.5]
#
# The Parameter Synthesis problem accepts two constants: 
#  - population_size: the number of individuals (S+I+R)
#  - infectious_days: the number of infected days before recovery
#
#  The scenario configuration also supports use of:
#  - epochs: the day index ranges corresponding to each beta
#  - cached_parameter_space: a precomputed Parameter Space encoded as JSON
#  - real_time_plotting: whether to render the solver's progress


gromet_file1 = "bar"
result1 : ParameterSynthesisScenarioResult = Funman().solve(
    ParameterSynthesisScenario(
        [Parameter("beta_0", lb=0.0, ub=0.5), Parameter("beta_1", lb=0.0, ub=0.5)], 
        gromet_file1,
        config = {
            "epochs": [(0, 20), (20, 40)],
            "population_size": 1002,
            "infectious_days": 14.0, 
            "cached_parameter_space" : "ps1.json",
            "real_time_plotting": False
        }
    ))

result1.parameter_space.plot()

In [ ]:
# This second use of Funman evaluates the model after changing the population
# size (10x) and infectious days (0.5x) constants.
   
gromet_file2 = "baz"
result2 : ParameterSynthesisScenarioResult = Funman().solve(
    ParameterSynthesisScenario(
        [Parameter("beta_0", lb=0.0, ub=0.5), Parameter("beta_1", lb=0.0, ub=0.5)],
        gromet_file2,
        config = {
            "epochs": [(0, 20), (20, 40)],
            "population_size": 10002,
            "infectious_days": 7.0, 
            "cached_parameter_space" : "ps2.json",
            "real_time_plotting": False
        }
    ))

result2.parameter_space.plot()

In [ ]:
# Funman produces two parameter spaces for beta_0 and beta_1.  The parameter
# spaces overlap (i.e., have a non-empty intersection) and indicate values for
# the betas that are robust to changes in the model constants.  Similarly, the
# distinct regions (i.e., symmetric difference) denote parameter values where
# the models behave differently.
#
# The plots below illustrate:
# - The parameter space for the original model constants.
# - The parameter space for the modified model constants.
# - The intersection of the parameter spaces above.
# - The symmetric difference of the parameter spaces above.
#
# The interpretation of the intersection is that the feasible values for beta
# denote interventions that scale with population size and the number of
# infectious days.  These values warrant further investigation by the analyst.
# The interpretation of the symmetric difference is that the feasible values for
# beta denote interventions that will not scale with population size and the
# number of infectious days.  These values may be discarded by the analyst.

intersection = ParameterSpace.intersect(result1, result2)
symmetric_difference = ParameterSpace.symmetric_difference(result1, result2)

result1.parameter_space.plot()
plt.show() ## comment this line out to get result1 and result2 on same figure
result2.parameter_space.plot()
plt.show()
intersection.plot()
plt.show()
symmetric_difference.plot()
plt.show()